**Dependencies**

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r"C:\Users\T 480\MALD\MELD\train\Textdataset.csv")

In [3]:
df

Speaker   Emotion Sentiment  Dialogue_ID  Utterance_ID  Season  \
0            Chandler   neutral   neutral            0             0       8   
1     The Interviewer   neutral   neutral            0             1       8   
2            Chandler   neutral   neutral            0             2       8   
3     The Interviewer   neutral   neutral            0             3       8   
4            Chandler  surprise  positive            0             4       8   
...               ...       ...       ...          ...           ...     ...   
9984         Chandler   neutral   neutral         1038            13       2   
9985             Ross   neutral   neutral         1038            14       2   
9986             Joey  surprise  positive         1038            15       2   
9987              All   neutral   neutral         1038            16       2   
9988             Joey       joy  positive         1038            17       2   

      Episode     StartTime       EndTime  \
0          21  00:16:16,059  00:16:21,731   
1          21  00:16:21,940  00:16:23,442   
2          21  00:16:23,442  00:16:26,389   
3          21  00:16:26,820  00:16:29,572   
4          21  00:16:34,452  00:16:40,917   
...       ...           ...           ...   
9984        3  00:00:48,173  00:00:50,799   
9985        3  00:00:51,009  00:00:53,594   
9986        3  00:01:00,518  00:01:03,520   
9987        3  00:01:05,398  00:01:07,274   
9988        3  00:01:08,401  00:01:12,071   

                                    Processed_Utterance  Duration  \
0     ['also', 'point', 'person', 'transition', 'sys...     5.672   
1                                      ['hand', 'full']     1.502   
2                                                    []     2.947   
3                     ['talk', 'little', 'bit', 'duty']     2.752   
4                                     ['duty', 'right']     6.465   
...                                                 ...       ...   
9984                                                 []     2.626   
9985    ['got', 'uh', 'joey', 'woman', 'adam', 'apple']     2.585   
9986                        ['guy', 'messing', 'right']     3.002   
9987                                           ['yeah']     1.876   
9988          ['good', 'one', 'second', 'like', 'whoa']     3.670   

                  Processed_Utterance_Str  
0     also point person transition system  
1                               hand full  
2                                     NaN  
3                    talk little bit duty  
4                              duty right  
...                                   ...  
9984                                  NaN  
9985         got uh joey woman adam apple  
9986                    guy messing right  
9987                                 yeah  
9988            good one second like whoa  

[9989 rows x 12 columns]

In [4]:
!pip install transformers torch scikit-learn pandas


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch

# Load your dataset
#df = pd.read_csv('your_dataset.csv')  # Replace with your actual dataset path

# Ensure that 'Processed_Utterance_Str' is of type string
df['Processed_Utterance_Str'] = df['Processed_Utterance_Str'].astype(str)

# Drop rows with missing values in 'Processed_Utterance_Str' or 'Emotion' columns
df = df.dropna(subset=['Processed_Utterance_Str', 'Emotion'])

# Map string labels to integer indices
label_map = {label: idx for idx, label in enumerate(df['Emotion'].unique())}
df['Emotion'] = df['Emotion'].map(label_map)

# Extract texts and labels
texts = df['Processed_Utterance_Str'].tolist()
labels = df['Emotion'].tolist()

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# Create a custom dataset class
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Convert labels to long tensor
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\T 480\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
